In [1]:
## Cells with personal data as part of a string or input values have either been scrubbed and replaced with "ADDRESS" 
## or deleted

In [5]:
import json
import pandas as pd
from collections import Counter
from datetime import date, time, datetime
from matplotlib import pyplot as plt

In [6]:
day = date.today().strftime("%B %Y")
# months = ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY', 'AUGUST',
#             'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER']
home_count=[]
work_count1=[]
work_count2=[]
full_year=[]

In [7]:
full_year=[]
for month in months:
    f = open(f'C:/Users/jspit/OneDrive/Documents/Takeout/Location History/Semantic Location History/2020/2020_{month}.json')
    data=json.load(f)
    address_list=[]
    
    for items in data.values():
        for item in items:
            for k in item.values():
                try:
                    address_list.append(k['location']['address'].split('\n')[0])
                except:
                    None
                    
    loc_count = dict(Counter(address_list))
    print(loc_count)
    full_year.append({"Month": month, "Locations": loc_count})
    try:
        home_count.append(loc_count['ADDRESS'])
        work_count1.append(loc_count['ADDRESS'])
        work_count2.append(loc_count['ADDRESS'])
    except:
        work_count1.append(0)
        work_count2.append(0)
print(work_count1)
print(loc_count)
print(full_year)

NameError: name 'months' is not defined

In [8]:
df = pd.json_normalize(full_year)
df.fillna(0, inplace=True)
df.reset_index()
df.set_index('Month', inplace=True)

for column, data in df.iteritems():
    if not any(i > 2 for i in data.values):
        del df[column]
df.astype(int)
for column, data in df.iteritems():
    df.rename({column: column.strip('Locations.')}, inplace=True)
df
pd.set_option('display.max_columns',25)
df

KeyError: "None of ['Month'] are in the columns"

In [ ]:
plt.bar(months, df_places['Vons'], label='Vons')
plt.bar(months, df_places['Friend House'], label="friend's house")
plt.bar(months, df_places['Trader Joes'], label = "Trader Joes")
plt.bar(months, df_places['Hiking trail'], label='Hiking Trail')
plt.grid(True)
plt.xticks(months, rotation=30)
plt.title('Frequently Visited Places - 2020')
plt.ylabel('Frequency Count')
plt.legend()

In [ ]:
df = pd.json_normalize(full_year)

dfx = df.loc[:,['Month', 'Locations.ADDRESS', 'Locations.ADDRESS','Locations.ADDRESS','Locations.ADDRESS']]
dfx.rename(columns={'Locations.ADDRESS':'Home','Locations.ADDRESS':'Work Location 1a', 'Locations.ADDRESS':'Work Location 1b',
                    'Locations.ADDRESS': 'Work Location 2' }, inplace=True)

dfx = dfx.fillna(0)
dfx = dfx.astype({'Work Location 1a': 'int64', 'Work Location 1b': 'int64'})

dfx['Work Location 1'] = dfx['Work Location 1a'] + dfx['Work Location 1b']
dfx

In [ ]:
dfx1 = pd.DataFrame(dfx)

del dfx1['Work Location 1a']
del dfx1['Work Location 1b']

In [ ]:
dfx1
dfx2 = dfx1[['Month', 'Work Location 1', 'Work Location 2']]
dfx2

In [ ]:
plt.plot(months,home_count, label='Home')
plt.plot(months,dfx['Work Location 1'], label='Work Location 1')
plt.plot(months,dfx['Work Location 2'], label='Work Location 2')
plt.xlabel('Month')
plt.xticks(months, rotation=30)
plt.ylabel('Frequency of Arrivals')
plt.title('Quantity of Times Arriving Home and Work Per Month - 2020')
plt.legend(bbox_to_anchor=(1, 1))
plt.grid(True)
plt.rcParams['figure.figsize'] = (9,5)
plt.show()
plt.tight_layout()

In [ ]:
from vincenty import vincenty
dist_list=[]
dist_month = {}
for month in months:
    f = open(f'C:/Users/jspit/OneDrive/Documents/Takeout/Location History/Semantic Location History/2020/2020_{month}.json')
    data=json.load(f)
    for items in data['timelineObjects']:
        try:
            item = items['activitySegment']
            start_coord = (int(item['startLocation']['latitudeE7'])/1e7, int(item['startLocation']['longitudeE7'])/1e7)
            end_coord = (int(item['endLocation']['latitudeE7'])/1e7, int(item['endLocation']['longitudeE7'])/1e7)
            distance = vincenty(start_coord, end_coord, miles=True)
            dist_month = {"Month": month, "Distance":distance}
        except:
            None
        dist_list.append(dist_month)

df2 = pd.DataFrame(dist_list)

df2_grp = df2.groupby(['Month'])
dist_totals = df2_grp['Distance'].sum()
df3 = pd.DataFrame(dist_totals)
df3 = df3.rename(columns = {'Distance': 'Monthly Distance Sum'})
df3

In [ ]:
months = ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY', 'AUGUST',
          'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER']
val= []
for index, row in df3.iterrows():
    val.append(row['Monthly Distance Sum'])

dfsort = pd.DataFrame({'Month': [i for i in months], 'num':[0,1,2,3,4,5,6,7,8,9,10,11]})
df4 = pd.merge(df3, dfsort, on='Month')
df4=df4.sort_values('num')
plt1 = df4.plot(kind='bar',x='Month', y = 'Monthly Distance Sum', ylabel='Monthly Distance (mi)', title ='Monthly Distance Totals', figsize=(8,8))
plt1.get_legend().remove()


In [ ]:
newlist=[]
for month in months:
    f = open(f'C:/Users/jspit/OneDrive/Documents/Takeout/Location History/Semantic Location History/2020/2020_{month}.json')
    data=json.load(f)
    for items in data['timelineObjects']:
        try:
            item = items['activitySegment']
            start_coord = (int(item['startLocation']['latitudeE7'])/1e7, int(item['startLocation']['longitudeE7'])/1e7)
            end_coord = (int(item['endLocation']['latitudeE7'])/1e7, int(item['endLocation']['longitudeE7'])/1e7)
            distance = vincenty(start_coord, end_coord, miles=True)
            travtype = item['activityType']
            dist_month = {"Month": month, "Distance":distance, "Activity Type": travtype}
        except:
            None
        newlist.append(dist_month)

df5 = pd.DataFrame(newlist)
df5
df5_grp = df5.groupby(['Month', 'Activity Type'])
act_totals = df5_grp['Distance'].sum()
act_totals
df6 = pd.DataFrame(act_totals)

df5_grp2 = df6.groupby(['Month', 'Activity Type'])
activity_totals = df5_grp2['Distance'].sum()

df5 = pd.DataFrame(activity_totals)
df5

In [ ]:
df5_grp2 = df5.groupby(['Activity Type'])
activity_totals = df5_grp2['Distance'].sum()
df5 = pd.DataFrame(activity_totals)
df5
df5.reset_index(inplace=True)
df5.sort_values(by='Distance', ascending=False)

In [ ]:
labels= ['', 'FLYING', '', 'IN_PASSENGER_VEHICLE', '', '', 'SKIING', 'WALKING']
x = [i for i in df5['Distance']]
explode=[0, 0.5, 0.1,0.4, 0.1, 0.1, 0.5, 0.7]
y = plt.pie(x, labels=labels, autopct='%1.1f%%', radius=2, explode=explode)